In [9]:
import numpy as np 
import matplotlib.pyplot as plt 

dict = np.load("outputs/bar_6_4(0)_6_600_psi_MM_(S)(PR)(r).npy",allow_pickle='TRUE').item()
dict = list(dict.values())

np.std(dict)

0.3391201771109469

In [31]:
import numpy as np 

L =9 
m =6
toggle_IL = True 

for i in np.arange(L):

        if toggle_IL==False:

            if i % 2 ==0:
                print("AA CL")
            elif i % 2 ==1:
                print("NN CL")
        
        if toggle_IL==True:

            # difference between number of input layers and number of target qubits:
            del_IL = m - (1+ L //3) 

            if del_IL <= 0:
                if i % 3 ==0:
                    print("AA CL")
                elif i % 3 ==1:
                   print("NN CL")
                elif i % 3 ==2:
                    # alternate between layers with control states 0 and 1 and add shifts
                    print(f"IL (shift={i//3 +1}; ctrl={i %2})")
            else: 
                if i % 3 ==0:
                     print("AA CL")
                elif i % 3 ==1:
                      print("NN CL")
                elif i % 3 ==2:
                    for j in np.arange(del_IL // (L //3) +2):
                        shift = (i // 3) +1 + j * (L //3)
                        #print(shift)
                        if shift <= m-1:
                            print(f"IL (shift={shift}; ctrl={(shift-1) %2})")


AA CL
NN CL
IL (shift=1; ctrl=0)
IL (shift=4; ctrl=1)
AA CL
NN CL
IL (shift=2; ctrl=1)
IL (shift=5; ctrl=0)
AA CL
NN CL
IL (shift=3; ctrl=0)
